# CTA Analysis - Visualizations
---
## Outline
This is part two of a two part analysis of the CTA Ridership data. In this section, I will focus on developing insights from the data.

**Database model:**
![title](database/cta_bus_ridership_database_model.png)

In [87]:
import collections
import itertools
import sqlite3
import pandas as pd
import numpy as np

# import visualizations libraries
import folium
import matplotlib.pyplot as plt
import seaborn as sns
from folium import plugins

from IPython.core.display import display, HTML
from IPython.display import IFrame

%matplotlib inline

In [3]:
# helper functions for sqlite3 
def connect(sqlite_file):
    """
    Make connection to an SQLite database file.
    """
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    return conn, c

def close(conn):
    """ 
    Commit changes and close connection to the database.
    """
    conn.close()
    
def table_col_info(cursor, table_name, print_out=False):
    """
    Returns a list of tuples with column informations:
    (id, name, type, notnull, default_value, primary_key)
    """
    cursor.execute('PRAGMA TABLE_INFO({})'.format(table_name))
    info = cursor.fetchall()

    if print_out:
        print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
        for col in info:
            print(col)
    return info

In [4]:
# sql_db is my connection and sql_command is my cursor
sql_db, sql_command = connect('database/cta_bus_ridership.db')


In [7]:
stop_most_routes = """
                    SELECT br.stop_id,
                           bs.on_street,
                           bs.cross_street,
                           COUNT(*) AS number_routes 
                    FROM bus_stops as bs
                    JOIN bus_routes as br ON bs.stop_id = br.stop_id
                    GROUP BY br.stop_id 
                    ORDER BY number_routes DESC LIMIT 10;
                   """
test = pd.read_sql_query(stop_most_routes, sql_db)
test


,stop_id,on_street,cross_street,number_routes
0,1106,MICHIGAN,WASHINGTON,14
1,73,MICHIGAN,betw. VAN BUREN & CONGRES,13
2,1103,MICHIGAN,HUBBARD (WRIGLEY BLDG.),13
3,1120,MICHIGAN,SOUTH WATER,13
4,1122,MICHIGAN,HUBBARD (TRIBUNE BLDG.),13
5,1100,MICHIGAN,ERIE,12
6,1101,MICHIGAN,OHIO,12
7,1102,MICHIGAN,ILLINOIS,12
8,1105,MICHIGAN,betw. LAKE & RANDOLPH,12
9,1123,MICHIGAN,GRAND,12


In [9]:
route_most_stops = """
                    SELECT br.route,
                           bs.on_street,
                           COUNT(*) AS total_stops
                    FROM bus_routes as br 
                    JOIN bus_stops as bs ON br.stop_id = bs.stop_id
                    GROUP BY br.route 
                    ORDER BY total_stops DESC LIMIT 10;
                   """
test = pd.read_sql_query(route_most_stops, sql_db)
test

,route,on_street,total_stops
0,9,ASHLAND,273
1,49,WESTERN,242
2,151,ADAMS,221
3,8,HALSTED,220
4,3,MICHIGAN,213
5,82,HOMAN,209
6,62,STATE,206
7,79,79TH STREET,206
8,N5,S. SHORE DRIVE,206
9,4,COTTAGE GROVE,202


In [75]:
bus_ridership_coords = """
                        SELECT br.stop_id,
                               br.boardings,
                               br. alightings,
                               bs.latitude,
                               bs.longitude 
                        FROM bus_stops as bs
                        JOIN bus_ridership as br ON bs.stop_id = br.stop_id;
                       """

bus_ridership_coords = pd.read_sql_query(bus_ridership_coords, sql_db)

# create to (n, 2) nd-array format for plotting and heatmap
stopArray = bus_ridership_coords[['latitude', 'longitude']].values

# create folium map of chicago
chicago_map = folium.Map([41.8781, -87.6298], zoom_start=10.9)

# mark each stop as a point on map
for latitude, longitude in stopArray:
    folium.CircleMarker([latitude, longitude],
                        radius = 0.5,
                        color="#3db7e4", 
                       ).add_to(chicago_map)

# plot heatmap
chicago_map.add_child(plugins.HeatMap(stopArray, radius=15, min_opacity=0.7))

# save heatmap
chicago_map.save('visualizations/bus_stops_heatmap.html')


In [89]:
IFrame(src='visualizations/bus_stops_heatmap.html', width=700, height=600)

In [82]:
# %%HTML
# <iframe src='visualizations/bus_stops_heatmap.html' width=1000 height=800></iframe>

In [62]:
import os
import time
from selenium import webdriver

browser = webdriver.Firefox()
browser.get('file:///{}{}'.format(os.getcwd(),'/visualizations/bus_stops_heatmap.html'))
time.sleep(5)  #Give the map tiles some time to load
browser.save_screenshot('visualizations/bus_stops_heatmap.png')
browser.quit()